## Importamos las librerias con las que vamos a trabajar

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [20]:
# 1) Cargar el dataset desde CSV
df = pd.read_csv("Crop_recommendation.csv")

# 2) Separar características (X) y etiqueta (y)
#    Asumiendo que las columnas son: N, P, K, temperature, humidity, ph, rainfall, label
X = df[["N", "P", "K", "temperature", "humidity", "ph", "rainfall"]]
y = df["label"]

# 3) Convertir las etiquetas (strings de cultivos) a números
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [21]:
# 4) División 70 % entrenamiento / 30 % prueba (estratificada para mantener proporción de clases)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.30,
    random_state=42,
    stratify=y_encoded
)

In [22]:
# 5) Definir dos clasificadores SVM multiclasé:
#    - Uno con kernel LINEAL
#    - Otro con kernel RBF
svm_linear = SVC(kernel="linear", random_state=42)
svm_rbf    = SVC(kernel="rbf", random_state=42)

In [23]:
# 6) Validación cruzada (cv=5) sobre el conjunto de entrenamiento
#    Usamos cross_val_score para obtener las puntuaciones en 5 folds
cv_scores_linear = cross_val_score(svm_linear, X_train, y_train, cv=5)
cv_scores_rbf    = cross_val_score(svm_rbf,    X_train, y_train, cv=5)

print("=== Validación Cruzada (5‐fold) en datos de ENTRENAMIENTO ===\n")

print("SVM LINEAL:")
print(" Scores por fold:       ", cv_scores_linear)
print(" Media de accuracy:     {:.4f}".format(cv_scores_linear.mean()))
print(" Desviación estándar:   {:.4f}".format(cv_scores_linear.std()))
print()

print("SVM RBF:")
print(" Scores por fold:       ", cv_scores_rbf)
print(" Media de accuracy:     {:.4f}".format(cv_scores_rbf.mean()))
print(" Desviación estándar:   {:.4f}".format(cv_scores_rbf.std()))
print("\n" + "="*60 + "\n")

=== Validación Cruzada (5‐fold) en datos de ENTRENAMIENTO ===

SVM LINEAL:
 Scores por fold:        [0.98051948 0.98376623 0.98051948 0.98051948 0.98051948]
 Media de accuracy:     0.9812
 Desviación estándar:   0.0013

SVM RBF:
 Scores por fold:        [0.98376623 0.96103896 0.97077922 0.97077922 0.96753247]
 Media de accuracy:     0.9708
 Desviación estándar:   0.0074




In [ ]:
# Definir el rango de valores para gamma
param_grid = {
    'gamma': [0.001, 0.01, 0.1, 1, 10]
}

# GridSearchCV con validación cruzada de 5 folds
grid_search = GridSearchCV(estimator=svm_rbf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajustar en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mostrar el mejor valor de gamma
print("Mejor gamma encontrado:", grid_search.best_params_['gamma'])
print("Mejor puntuación (accuracy media CV): {:.4f}".format(grid_search.best_score_))

Mejor gamma encontrado: 0.001
Mejor puntuación (accuracy media CV): 0.9844


In [25]:
# 7) Entrenar cada modelo con TODO el set de entrenamiento
svm_linear.fit(X_train, y_train)
best_svm_rbf = grid_search.best_estimator_

In [26]:
# 8) Hacer predicción sobre el conjunto de prueba (30 %)
y_pred_linear = svm_linear.predict(X_test)
y_pred_rbf    = best_svm_rbf.predict(X_test)

In [27]:
# 9) Calcular accuracy en prueba
accuracy_linear = accuracy_score(y_test, y_pred_linear)
accuracy_rbf    = accuracy_score(y_test, y_pred_rbf)

In [28]:
print("=== Resultados en CONJUNTO de PRUEBA (30 %) ===\n")
print(f"SVM LINEAL: Test Accuracy = {accuracy_linear:.4f}")
print(f"SVM RBF:    Test Accuracy = {accuracy_rbf:.4f}")
print("\n" + "="*60 + "\n")

# 10) Mostrar reporte de clasificación (precision / recall / f1-score) para cada kernel
print("Reporte de clasificación para SVM LINEAL:\n")
print(classification_report(
    y_test, 
    y_pred_linear, 
    target_names=label_encoder.classes_,
    zero_division=0
))

print("Reporte de clasificación para SVM RBF:\n")
print(classification_report(
    y_test, 
    y_pred_rbf, 
    target_names=label_encoder.classes_,
    zero_division=0
))

=== Resultados en CONJUNTO de PRUEBA (30 %) ===

SVM LINEAL: Test Accuracy = 0.9894
SVM RBF:    Test Accuracy = 0.9894


Reporte de clasificación para SVM LINEAL:

              precision    recall  f1-score   support

       apple       1.00      1.00      1.00        30
      banana       1.00      1.00      1.00        30
   blackgram       0.94      1.00      0.97        30
    chickpea       1.00      1.00      1.00        30
     coconut       1.00      1.00      1.00        30
      coffee       1.00      1.00      1.00        30
      cotton       0.94      1.00      0.97        30
      grapes       1.00      1.00      1.00        30
        jute       0.91      1.00      0.95        30
 kidneybeans       1.00      1.00      1.00        30
      lentil       1.00      1.00      1.00        30
       maize       1.00      0.93      0.97        30
       mango       1.00      1.00      1.00        30
   mothbeans       1.00      0.93      0.97        30
    mungbean       1.00  